# `key`, `as` and conflict

in a dict, it is easy to change a model by overwriting its entry such as:

In [57]:
import numpy as np
from pprint import pprint
import modeldag

# a default model with -10<a<0 and 100<b<110
initial_model = {"a": {"func": np.random.uniform, "kwargs": {"low": -10, "high":0}},
                 "b": {"func": np.random.uniform, "kwargs": {"low": 100, "high":110}},
                }
# update the model by changing its a entry
update_model = {"a": {"func": np.random.uniform, "kwargs": {"low": 50, "high":60}}}

# such that the final model has "b" from the initial and "a" from the update:
model = initial_model | update_model
pprint(model)

{'a': {'func': <built-in method uniform of numpy.random.mtrand.RandomState object at 0x107872d40>,
       'kwargs': {'high': 60, 'low': 50}},
 'b': {'func': <built-in method uniform of numpy.random.mtrand.RandomState object at 0x107872d40>,
       'kwargs': {'high': 110, 'low': 100}}}


but with the "as" options, this can but lost. 

**Fortunately, ModelDAG checks this while loading the model.**
***
*There is however several cases or interest*
***

## Single `as=name` case

This is the simplest case, key of the updating dict does not match with that from the initial dict, but the `as` makes it so.

In [58]:
# a default model with -10<a<0 and 100<b<110
initial_model = {"a": {"func": np.random.uniform, "kwargs": {"low": -10, "high":0}},
                 "b": {"func": np.random.uniform, "kwargs": {"low": 100, "high":110}},
                }

# update the model by changing its a entry
update_model = {"new_a": {"func": np.random.uniform, 
                          "kwargs": {"low": 50, "high":60},
                          "as":"a" # this will lead to 'a' in the final dataframe, so effectively overwriting 'a'
                         }
               }

# such that the final model has "b" from the initial and "a" from the update:
model = initial_model | update_model
pprint(model)

{'a': {'func': <built-in method uniform of numpy.random.mtrand.RandomState object at 0x107872d40>,
       'kwargs': {'high': 0, 'low': -10}},
 'b': {'func': <built-in method uniform of numpy.random.mtrand.RandomState object at 0x107872d40>,
       'kwargs': {'high': 110, 'low': 100}},
 'new_a': {'as': 'a',
           'func': <built-in method uniform of numpy.random.mtrand.RandomState object at 0x107872d40>,
           'kwargs': {'high': 60, 'low': 50}}}


There seem to be a conflict, but modeldag has tools that checks in 'as' to clean the input model

It knows here as "new_a" should be assumed as 'a' and therefore behaves as such

In [59]:
dag = modeldag.ModelDAG(model)

In [60]:
pprint(dag.model,  sort_dicts=False)

{'b': {'func': <built-in method uniform of numpy.random.mtrand.RandomState object at 0x107872d40>,
       'kwargs': {'low': 100, 'high': 110}},
 'new_a': {'func': <built-in method uniform of numpy.random.mtrand.RandomState object at 0x107872d40>,
           'kwargs': {'low': 50, 'high': 60},
           'as': 'a'}}


In [61]:
dag.draw(2)

,b,a
0,106.309783,57.430408
1,102.372642,57.531347


***
## `as=list` cases

Things are most complex with `as` is used to specify that several entries as used. and again several cases exist


### joined draw overwrites former keys

In [62]:
def joined_draw(size, alpha=3., beta=1.5, **kwargs):
    """ """
    a = np.random.uniform(size=size, **kwargs)
    b = a*alpha + beta
    return a, b

# a default model with -10<a<0 and 100<b<110
initial_model = {"a": {"func": np.random.uniform, "kwargs": {"low": -10, "high":0}},
                 "b": {"func": np.random.uniform, "kwargs": {"low": 100, "high":110}},
                }

# Now, new model makes that a and b are drawn simultaneously.
update_model = {"a_and_b": {"func": joined_draw, 
                             "kwargs": {"low":0, "high":2}, 
                             "as": ["a", "b"]}
                }

model = initial_model | update_model
pprint(model, sort_dicts=False)

{'a': {'func': <built-in method uniform of numpy.random.mtrand.RandomState object at 0x107872d40>,
       'kwargs': {'low': -10, 'high': 0}},
 'b': {'func': <built-in method uniform of numpy.random.mtrand.RandomState object at 0x107872d40>,
       'kwargs': {'low': 100, 'high': 110}},
 'a_and_b': {'func': <function joined_draw at 0x1345722a0>,
             'kwargs': {'low': 0, 'high': 2},
             'as': ['a', 'b']}}


**In that case:** the solution is simple, ModelDAG knows it has to overwrite both `a` and `b` with the new `a_and_b`

In [63]:
dag = modeldag.ModelDAG(model)
dag.model

{'a_and_b': {'func': <function __main__.joined_draw(size, alpha=3.0, beta=1.5, **kwargs)>,
  'kwargs': {'low': 0, 'high': 2},
  'as': ['a', 'b']}}

Say now that only `b` is updated by the joined draw, but now `a`

In [64]:
# Now, new model makes that a and b are drawn simultaneously.
update_model = {"b_and_c": {"func": joined_draw, 
                             "kwargs": {"low":0, "high":2}, 
                             "as": ["b","c"]}
                }


Same, it is easy for ModelDAG to know what to do:
 - `a` is left unchanged
 - `b` is replaced by the b_and_c draw.

In [65]:
dag = modeldag.ModelDAG( initial_model | update_model )
dag.model

{'a': {'func': <function RandomState.uniform>,
  'kwargs': {'low': -10, 'high': 0}},
 'b_and_c': {'func': <function __main__.joined_draw(size, alpha=3.0, beta=1.5, **kwargs)>,
  'kwargs': {'low': 0, 'high': 2},
  'as': ['b', 'c']}}

### new draw overwrites former as=list

Say you and up with a model dict that has no obvious solutions, for instance

In [66]:
# a default model with -10<a<0 and 100<b<110
complex_model = {"a": {"func": np.random.uniform, 
                       "kwargs": {"low": -10, "high":0}},
                 
                 "a_and_b": {"func": joined_draw, 
                             "kwargs": {"low":0, "high":2}, 
                             "as": ["a", "b"]},
                 "b": {"func": np.random.uniform, "kwargs": {"low": 100, "high":110}},
                }

here `a_and_b` is expected to replace `a`. That is ok. 

But after, `b` wants to replace existing `b` drawn as part of `a_and_b`. But then what about `a` ? Since both `a` and `b` are supposed to be drawn together, it does not make sense to *just* replace `b`.

**In such a case**, `ModelDAG` will raise a `ValueError` but default

In [67]:
dag = modeldag.ModelDAG( complex_model )

ValueError: new key_or_as='b' cannot replace that from a_and_b ('as': ['a', 'b'])

You can however force it to accept this by specifying as_conflict='warn' or 'skip'.

In [70]:
dag = modeldag.ModelDAG( complex_model, as_conflict="warn") # use skip to ignore the warning.

/Users/rigault/miniforge3/envs/ztfdc/lib/python3.11/site-packages/modeldag/tools.py:65: UserWarning: new key_or_as='b' cannot replace that from a_and_b ('as': this_as['as_orig']). This is skiped. Potentially leads to conflict.
  warnings.warn(f"new {key_or_as=} cannot replace that from {this_as['input_key']} ('as': this_as['as_orig']). This is skiped. Potentially leads to conflict.")


In [71]:
dag.draw(3)

,a,b
0,1.651262,104.747112
1,0.028661,106.084687
2,1.160684,103.223104


In that case `b` will be overwritten and `a` unchanged.